# Build MpipiGG forcefield
##### Last updated 2024-01-13

This notebook walks through the steps for editing the original Mpipi parameters to construct the Mpipi-GG forcefield.  Note that the aliphatic context residues are NOT used in the ALBATROSS paper.

In [2]:
import finches
import pickle

# import dedicated functions for this conversion
from finches.data.mPiPi import mPiPi_GGv1_modification_fxns as mpipi_gg_update

In [6]:
# define all (standard) residues that are supported by Mpipi-GG (i.e. those we can pass in from finches)
ALL_RESIDUES_TYPES = ['A','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y','C','U']

def sanity_check_revised_dictionary(indict, all_allowed_res=ALL_RESIDUES_TYPES):
    """
    Sanity check function to ensure the update functions we call return a valid
    dictionary that meets the specification. Note that Mpipi-GG actually defines
    some additional residues types (lower-case letters) for context-dependent
    aliphatic residues, but we don't validate those here, assuming/hoping that
    the function which created them is responsible for their validation.

    Parameters
    -------------
    indict : dict
        Dictionary that has a [A1][A2] mapping structure with the full, 
        redundant matrix of inter-residue/nucleotide parameters for whatever
        the matrix passed offers.

    all_allowed_res : list

    """
    if not isinstance(indict, dict):
        raise Exception('Passed function did not generate a dictionary')
        
    # check all possible combos return floatable values
    for aa1 in all_allowed_res:
        for aa2 in all_allowed_res:
            try:
                _x = float(indict[aa1][aa2])
            except Exception as e:
                raise Exception(f'Dictionary could not return sigma values between {aa1} and {aa2}.\nUpdate dictionary generated is: {str(indict)}.Error below:\n({str(e)}\n')


In [7]:
# define the location where the original Mpipi parameters are located
data_prefix = finches.get_data('mPipi')


### Step 1: Read in default parameters
Read in the default Mpipi parameters

In [8]:
with open(f'{data_prefix}/sigma.pickle', 'rb') as fh:
    SIGMA_ALL_Mpipi_OG = pickle.load(fh)
        
with open(f'{data_prefix}/epsilon.pickle', 'rb') as fh:
    EPSILON_ALL_Mpipi_OG = pickle.load(fh)
    
with open(f'{data_prefix}/nu.pickle', 'rb') as fh:
    NU_ALL_Mpipi_OG = pickle.load(fh)    
    
with open(f'{data_prefix}/mu.pickle', 'rb') as fh:
    MU_ALL_Mpipi_OG = pickle.load(fh)        
    
with open(f'{data_prefix}/charge.pickle', 'rb') as fh:
    CHARGE_ALL_Mpipi_OG = pickle.load(fh)    


In [9]:
with open(f'{data_prefix}/sigma.pickle', 'rb') as fh:
    SIGMA_ALL = pickle.load(fh)
        
with open(f'{data_prefix}/epsilon.pickle', 'rb') as fh:
    EPSILON_ALL = pickle.load(fh)
    
with open(f'{data_prefix}/nu.pickle', 'rb') as fh:
    NU_ALL = pickle.load(fh)    
    
with open(f'{data_prefix}/mu.pickle', 'rb') as fh:
    MU_ALL = pickle.load(fh)        
    
with open(f'{data_prefix}/charge.pickle', 'rb') as fh:
    CHARGE_ALL = pickle.load(fh)    


In [10]:
# FIRST update RNA. This MUST happen first so we continue with a full updated set of parameters
CHARGE_ALL, MU_ALL, NU_ALL, EPSILON_ALL, SIGMA_ALL = mpipi_gg_update.ADD_RNA_U_ALL(CHARGE_ALL, MU_ALL, NU_ALL, EPSILON_ALL, SIGMA_ALL)

sanity_check_revised_dictionary(MU_ALL)
sanity_check_revised_dictionary(NU_ALL)
sanity_check_revised_dictionary(EPSILON_ALL)
sanity_check_revised_dictionary(SIGMA_ALL)

In [11]:
# correct existing aliphatic residues. This MUST happen second, because those values are then used when initializing 
# the new context-dependent alphatic residues
CHARGE_ALL, MU_ALL, NU_ALL, EPSILON_ALL, SIGMA_ALL = mpipi_gg_update.CORRECT_aliphatic_aliphatic_ALL(CHARGE_ALL, MU_ALL, NU_ALL, EPSILON_ALL, SIGMA_ALL)
sanity_check_revised_dictionary(MU_ALL)
sanity_check_revised_dictionary(NU_ALL)
sanity_check_revised_dictionary(EPSILON_ALL)
sanity_check_revised_dictionary(SIGMA_ALL)

In [12]:
# Next we add in the new alphatic residues!
CHARGE_ALL, MU_ALL, NU_ALL, EPSILON_ALL, SIGMA_ALL = mpipi_gg_update.CREATE_new_aliphatic_residues_ALL(CHARGE_ALL, MU_ALL, NU_ALL, EPSILON_ALL, SIGMA_ALL)
sanity_check_revised_dictionary(MU_ALL)
sanity_check_revised_dictionary(NU_ALL)
sanity_check_revised_dictionary(EPSILON_ALL)
sanity_check_revised_dictionary(SIGMA_ALL)

In [13]:
# next we scale aliphatic groups in different contexts
EPSILON_ALL = mpipi_gg_update.SCALE_aliphatic_group_EPSILON(EPSILON_ALL)
sanity_check_revised_dictionary(EPSILON_ALL)


In [14]:
# next we update minor tweaks which include

## proline scaling
SIGMA_ALL = mpipi_gg_update.ENLARGE_Proline_SIGMA(SIGMA_ALL)
sanity_check_revised_dictionary(SIGMA_ALL)

## strengthening  G:S and G:G interaction strengths
EPSILON_ALL = mpipi_gg_update.STRENGTHEN_small_polar_EPSILON(EPSILON_ALL)
sanity_check_revised_dictionary(EPSILON_ALL)

## weakening E|D|R : Y|F|W interaction strength
EPSILON_ALL = mpipi_gg_update.WEAKEN_Aromatic_Charge_EPSILON(EPSILON_ALL)
sanity_check_revised_dictionary(EPSILON_ALL)




In [15]:
# finally print changes

In [16]:
print('Epsilon changes')
for idx1 in range(len(EPSILON_ALL)):
    for idx2 in range(idx1, len(EPSILON_ALL)): 
        AA1 = list(EPSILON_ALL.keys())[idx1]
        AA2 = list(EPSILON_ALL.keys())[idx2]

        
        if AA1 in EPSILON_ALL_Mpipi_OG and AA2 in EPSILON_ALL_Mpipi_OG:
            if not np.isclose(EPSILON_ALL[AA1][AA2], EPSILON_ALL_Mpipi_OG[AA1][AA2]):
                print(f'{AA1}-{AA2} (new-old): {np.round(EPSILON_ALL[AA1][AA2] -EPSILON_ALL_Mpipi_OG[AA1][AA2],3)}')
            
        

Epsilon changes
M-M (new-old): 0.01
M-A (new-old): 0.004
M-V (new-old): 0.046
M-L (new-old): 0.04
M-I (new-old): 0.051
G-G (new-old): 0.115
G-S (new-old): 0.094
R-Y (new-old): -0.365
R-W (new-old): -0.415
R-F (new-old): -0.327
A-A (new-old): -0.002
A-V (new-old): 0.04
A-L (new-old): 0.034
A-I (new-old): 0.045
D-Y (new-old): -0.158
D-W (new-old): -0.197
D-F (new-old): -0.15
E-Y (new-old): -0.16
E-W (new-old): -0.2
E-F (new-old): -0.152
V-V (new-old): 0.082
V-L (new-old): 0.076
V-I (new-old): 0.088
L-L (new-old): 0.07
L-I (new-old): 0.081
I-I (new-old): 0.093


In [17]:
print('Sigma changes')
for idx1 in range(len(SIGMA_ALL)):
    for idx2 in range(idx1, len(SIGMA_ALL)): 
        AA1 = list(EPSILON_ALL.keys())[idx1]
        AA2 = list(EPSILON_ALL.keys())[idx2]

        
        if AA1 in EPSILON_ALL_Mpipi_OG and AA2 in EPSILON_ALL_Mpipi_OG:
            if not np.isclose(SIGMA_ALL[AA1][AA2], SIGMA_ALL_Mpipi_OG[AA1][AA2]):
                print(f'{AA1}-{AA2} (new-old): {np.round(SIGMA_ALL[AA1][AA2] - SIGMA_ALL_Mpipi_OG[AA1][AA2],3)}')
        

Sigma changes
M-P (new-old): 2.024
G-P (new-old): 1.732
K-P (new-old): 2.056
T-P (new-old): 1.928
R-P (new-old): 2.086
A-P (new-old): 1.827
D-P (new-old): 1.919
E-P (new-old): 1.977
Y-P (new-old): 2.068
V-P (new-old): 1.972
L-P (new-old): 2.026
Q-P (new-old): 1.993
W-P (new-old): 2.123
F-P (new-old): 2.051
S-P (new-old): 1.851
H-P (new-old): 2.003
N-P (new-old): 1.935
P-P (new-old): 1.916
P-C (new-old): 1.902
P-I (new-old): 2.027


In [19]:
with open(f'{data_prefix}/sigma_ggv1.pickle', 'wb') as fh:
    pickle.dump(SIGMA_ALL, fh)
        
with open(f'{data_prefix}/epsilon_ggv1.pickle', 'wb') as fh:
     pickle.dump(EPSILON_ALL, fh)
    
with open(f'{data_prefix}/nu_ggv1.pickle', 'wb') as fh:
     pickle.dump(NU_ALL, fh)    
    
with open(f'{data_prefix}/mu_ggv1.pickle', 'wb') as fh:
     pickle.dump(MU_ALL, fh)        
    
with open(f'{data_prefix}/charge_ggv1.pickle', 'wb') as fh:
     pickle.dump(CHARGE_ALL, fh)    
